Imports

In [307]:
!pip install -r requirements.txt

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


In [308]:
import pandas as pd

Load datasets

In [309]:
def read_csv(filename):
    df = pd.read_csv(
        filename,
        header=0,                     # Use the first row as the header
        delimiter=';',                # Use semicolon as the main delimiter
        decimal=',',                  # Specify that commas are used as decimals
        quotechar='"',                # Handle quotes around strings
        skipinitialspace=True,        # Skip spaces after delimiters
    )
    df.set_index(df.columns[0], inplace=True)
    return df

airlines = read_csv('airlines.csv')
airports = read_csv('airports.csv')
planes = read_csv('planes.csv')
flights = read_csv('flights.csv')
weather = read_csv('weather.csv')

In [310]:
def check_for_nulls(df):
    res = {}
    for c in df.columns:
        if df[df[c].isnull()].shape[0] != 0:
            res[c] = df[df[c].isnull()].shape[0]
    return res

In [311]:
print("Nullable columns in airlines:", check_for_nulls(airlines))
print("Nullable columns in airports:", check_for_nulls(airports))
print("Nullable columns in planes:", check_for_nulls(planes))
print("Nullable columns in flights:", check_for_nulls(flights))
print("Nullable columns in weather:", check_for_nulls(weather))

Nullable columns in airlines: {}
Nullable columns in airports: {'tz': 48, 'dst': 48, 'tzone': 119}
Nullable columns in planes: {'year': 89}
Nullable columns in flights: {'dep_time': 10738, 'dep_delay': 10738, 'arr_time': 11453, 'arr_delay': 12534, 'tailnum': 1913, 'air_time': 12534}
Nullable columns in weather: {'temp': 25536, 'dewp': 25536, 'humid': 25536, 'wind_dir': 1220, 'wind_speed': 1033, 'wind_gust': 1033, 'precip': 24611, 'pressure': 25632, 'visib': 24}


Clean Datasets

In [312]:
def clear_airports(df):
    # I checked that tz is null = dst is null
    df = df.dropna(subset=['tz', 'dst'])
    for idx, row in df[df['tzone'].isnull()].iterrows():
        matching_row = df[(df['tz'] == row['tz']) & pd.notnull(df['tzone'])]
        if not matching_row.empty:
            df.at[idx, 'tzone'] = matching_row['tzone'].iloc[0]
        else:
            df = df.drop(idx)
    return df

In [313]:
def clear_planes(df):
    return df.dropna()

In [314]:
def clear_flights(df):
    # I checked that dep_time is null = dep_delay is null \subset arr_time is null \subset arr_delay is null \subset air_time is null
    df = df.dropna(subset=['arr_time', 'tailnum'])
    for idx, row in df[df['air_time'].isnull()].iterrows():
        df.at[idx, 'arr_delay'] = df['arr_time'].iloc[0] - df['sched_arr_time'].iloc[0]
        df.at[idx, 'air_time'] = df['arr_time'].iloc[0] - df['dep_time'].iloc[0]
    return df

In [315]:
def clear_weather(df):
    # I checked that precip is null \subset temp is null = dewp is null = humid is null \subset pressure is null
    df['temp'] = df['temp'].fillna(df['temp'].median())
    df['dewp'] = df['dewp'].fillna(df['dewp'].median())
    df['humid'] = df['humid'].fillna(df['humid'].median())
    df['wind_speed'] = df['wind_speed'].fillna(df['wind_speed'].mean())
    return df.ffill().bfill()

In [316]:
clean_airlines = airlines
clean_airports = clear_airports(airports.copy())
clean_planes = clear_planes(planes.copy())
clean_flights = clear_flights(flights.copy())
clean_weather = clear_weather(weather.copy())

In [317]:
print("Nullable columns in airlines:", check_for_nulls(clean_airlines))
print("Nullable columns in airports:", check_for_nulls(clean_airports))
print("Nullable columns in planes:", check_for_nulls(clean_planes))
print("Nullable columns in flights:", check_for_nulls(clean_flights))
print("Nullable columns in weather:", check_for_nulls(clean_weather))

Nullable columns in airlines: {}
Nullable columns in airports: {}
Nullable columns in planes: {}
Nullable columns in flights: {}
Nullable columns in weather: {}


In [318]:
print("Airports old vs new:", airports.shape, clean_airports.shape)
print("Planes old vs new:", planes.shape, clean_planes.shape)
print("Flights old vs new:", flights.shape, clean_flights.shape)
print("Weather old vs new:", weather.shape, clean_weather.shape)

Airports old vs new: (1251, 8) (1194, 8)
Planes old vs new: (4840, 9) (4751, 9)
Flights old vs new: (435352, 19) (423899, 19)
Weather old vs new: (26204, 15) (26204, 15)
